# Assignment 3 - Reinforcement Learning

## GridWorlds

This assignment involves finding optimal policies for two grid worlds (CliffWalking and WindyGridWorld) using SARSA and Q learning. Details about WindyGridWorld (Example 6.5) and CliffWalking (Example 6.6) can be found in the following link.
    
    http://incompleteideas.net/book/RLbook2020.pdf


You need gym (version 0.18) and numpy (version 1.20.1) for this assignment. The environment for both problems are provided. 

For Windy Grid World environemnt you also need the file 'WindyGridWorld.py'. 

### Task 1: Learning [5 Marks]

You only need to write the codes for SARSA and Q-learning algorithms. Then do the learning in both 'CliffWalking' and 'Windy Grid World' environments. 

### Task 2: Analysis [5 Marks]   

1. Calculate the average return across the episodes. It gives you a measure of the performance of the algorithm while learning.  

2. Calculate the return after convergence. It gives you a measure of the performance after the learning is completed. 

3. What do you observe from these results?

Install the necessary packages

In [ ]:
!pip install gym==0.22
!pip install numpy==1.20.1
!pip install tqdm 
!pip install pygame

# Task 1: Learning
## Task 1a: Learning in CliffWalking Environment

### Environment for CliffWalking

The board is a 4x12 matrix, with (using NumPy matrix indexing):
    [3, 0] as the start at bottom-left
    [3, 11] as the goal at bottom-right
    [3, 1..10] as the cliff at bottom-center

Each time step incurs -1 reward, and stepping into the cliff incurs -100 reward
and a reset to the start. If an action would take you off the grid, you remain in the previous state.
An episode terminates when the agent reaches the goal.


In [3]:

import gym
import numpy as np
from tqdm import tqdm 

env = gym.make('CliffWalking-v0') # Create the environment #render_mode="human"  human, ansi, 
env.reset() # reset environment to a new, random state
env.render() # Renders the environment for visualization

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T



Here _x_ is the location of the agent, *o* are possible places to go to, *C* is the cliff, and *T* is the target.

In [4]:
num_actions = env.action_space.n 
num_states = env.observation_space.n 

print("Number of actions: ", num_actions)
print("Number of states: ", num_states)

Number of actions:  4
Number of states:  48


In [5]:
action = 0 # Move up
a = env.step(action) # This is the function we use to interact with the environment
env.render() # Renders the environment for visualization

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T



In [6]:
# 0 -> UP, 1 -> RIGHT, 2 -> DOWN, 3 -> LEFT
env.reset()
import time
for action in [0, 1, 2, 3]:
    print("Action: ", action)
    time.sleep(1)
    next_state, reward, is_done, info = env.step(action)     # next_state, reward, is_done, info
    print("Next state: ", next_state)
    print("Reward: ", reward)
    print("Done: ",is_done)
    env.render()
env.reset()

Action:  0
Next state:  24
Reward:  -1
Done:  False
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Action:  1
Next state:  25
Reward:  -1
Done:  False
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  x  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Action:  2
Next state:  36
Reward:  -100
Done:  False
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

Action:  3
Next state:  36
Reward:  -1
Done:  False
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T



36

As you can see above, each non-terminal action has a reward of -1. 0 -> UP, 1 -> RIGHT, 2 -> DOWN, 3 -> LEFT. The moment the agent falls off the cliff the reward becomes -100 and the agent resets to the start.

In [7]:
# Initialize values 
num_episodes = 500
lr = 0.85 # Learning Rate
epsilon = 0.1

In [8]:
# Initialize Q function - a simplified version is used here 
# in reality the number of states may be unknown and all states may not be reachable 

# hint: use num_states as the key to a dictionary of lists
Q = np.zeros((num_states, num_actions))

In [9]:
def behavioral_policy(state, Q, num_actions, epsilon):
    # Implement the epsilon-greedy policy
    # Don't forget the epsilon-greedy idea
    probs =  np.ones(num_actions,dtype = float) * epsilon / num_actions
    best_action = np.argmax(Q[state])
    probs[best_action] += (1.0 - epsilon)
    
    action = np.argmax(np.random.multinomial(1, probs, size=1)[0])
    return action

In [10]:
# You can use this to check if your algorithm is correct
for i in range(10):
    print(behavioral_policy(0, Q, num_actions, 0.8))

0
0
0
2
1
0
0
0
3
3


### SARSA Learning 

In [11]:


def sarsa(env, Q, num_actions, num_episodes, epsilon, lr):
    # Given to students
    episode_length = [0] * num_episodes
    total_reward_episode = [0] * num_episodes

    for episode in tqdm(range(num_episodes)):
        state = env.reset()
        is_done = False
        # Implement SARSA
        action = behavioral_policy(state, Q, num_actions, epsilon)
        while not is_done:
            state_2, reward, done, _ = env.step(action)
            action_2 = behavioral_policy(state_2, Q, num_actions, epsilon)
            predict = Q[state, action]
            target = reward +  Q[state_2, action_2] # gamma = 1 
            Q[state, action] = Q[state, action] + lr * (target - predict)

            episode_length[episode] += 1
            total_reward_episode[episode] += reward
            state = state_2
            action = action_2
            is_done = done
    policy = {}
    # Write code here as well
    # Hint: use np.argmax

    return Q, policy, {"rewards": total_reward_episode, "length": episode_length}

In [12]:
# Run SARSA
optimal_sarsa_Q, sarsa_optimal_policy, sarsa_info = sarsa(env, Q, num_actions, num_episodes, epsilon, lr)
print("\nGridWorld SARSA Optimal policy: \n", sarsa_optimal_policy,sarsa_info)

100%|██████████| 500/500 [00:01<00:00, 309.41it/s]


GridWorld SARSA Optimal policy: 
 {} {'rewards': [-420, -1456, -43, -70, -63, -75, -85, -177, -86, -41, -420, -32, -32, -35, -119, -154, -35, -29, -26, -158, -23, -24, -27, -27, -29, -33, -25, -135, -33, -19, -22, -128, -20, -30, -29, -123, -26, -21, -31, -25, -124, -19, -228, -23, -23, -28, -21, -21, -27, -159, -25, -144, -22, -25, -25, -24, -35, -243, -21, -26, -164, -26, -27, -19, -32, -21, -25, -20, -19, -27, -29, -23, -31, -21, -19, -25, -167, -297, -36, -26, -41, -23, -30, -19, -28, -121, -19, -19, -18, -36, -18, -39, -23, -21, -19, -17, -19, -19, -21, -25, -17, -37, -156, -29, -33, -154, -20, -162, -27, -17, -22, -17, -17, -28, -118, -33, -17, -19, -17, -20, -20, -41, -25, -25, -26, -17, -22, -19, -23, -18, -19, -22, -20, -25, -44, -139, -23, -21, -22, -23, -21, -26, -19, -21, -23, -23, -25, -19, -43, -26, -45, -341, -245, -204, -53, -40, -46, -58, -32, -33, -21, -29, -67, -24, -42, -29, -27, -23, -23, -67, -123, -79, -61, -311, -32, -25, -17, -29, -47, -135, -24, -60, -43, -84

### Q-Learning

In [13]:
def q_learning(env, Q, num_actions, num_episodes, epsilon, lr):
    # Given to students
    episode_length = [0] * num_episodes
    total_reward_episode = [0] * num_episodes

   
    for episode in tqdm(range(num_episodes)):
        state = env.reset()
        is_done = False
        # Implemnt Q-Learning
        while not is_done:
            action = behavioral_policy(state, Q, num_actions, epsilon)
            next_state, reward, done, _ = env.step(action)
 
            total_reward_episode[episode] += reward
            episode_length[episode] += 1

            # Update Q values
            best_next_action = np.argmax(Q[next_state])   
            td_target = reward + Q[next_state][best_next_action] # gamma = 1 
            td_delta = td_target - Q[state][action]
            Q[state][action] += lr * td_delta

            is_done = done
            
    policy = {}
    # Write the code here

    return Q, policy, {"rewards": total_reward_episode, "length": episode_length}

In [14]:
# Run Q-Learning 

optimal_Q, q_optimal_policy, q_info = q_learning(env, Q, num_actions, num_episodes, epsilon, lr)
print("\nGridWorld Q-Learning Optimal policy: \n", q_optimal_policy,q_info)

100%|██████████| 500/500 [00:48<00:00, 10.28it/s]


GridWorld Q-Learning Optimal policy: 
 {} {'rewards': [-218, -4194, -8053, -2505, -9074, -192, -948, -729, -6499, -719, -3731, -19555, -550, -5958, -163, -868, -486, -5057, -6106, -5214, -4967, -3058, -7766, -6910, -3791, -3882, -2386, -9476, -8990, -2286, -5116, -1149, -1270, -9277, -2141, -281, -1742, -8840, -5638, -3263, -1118, -795, -395, -16653, -5231, -2650, -4712, -8228, -8181, -7497, -1092, -2288, -1557, -1717, -645, -1218, -3482, -488, -1328, -447, -1761, -9075, -1005, -908, -6924, -3722, -1709, -6785, -5272, -1992, -18444, -252, -2152, -7221, -2146, -11710, -1507, -2589, -1369, -9334, -2775, -819, -2599, -893, -1161, -1590, -4954, -580, -7025, -9712, -3820, -2143, -3440, -1371, -5123, -5801, -4035, -1678, -876, -2777, -6349, -8585, -1063, -722, -11778, -3771, -1698, -893, -4896, -5280, -542, -1206, -8504, -5778, -2540, -6905, -3906, -771, -7447, -532, -3311, -1568, -2310, -9871, -4801, -13948, -6580, -2609, -14846, -382, -12911, -824, -15505, -3599, -2695, -1832, -8562, -122

In [ ]:
# run this cell if you do not have the matplotlib library
# !pip install matplotlib
import matplotlib.pyplot as plt

In [ ]:
def plot_rate(episode_length, total_reward_episode, title):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].plot(episode_length)
    ax[0].set_title("Episode Length over time")
    ax[0].set(xlabel="Episode", ylabel="Length")
    ax[1].plot(total_reward_episode)
    ax[1].set_title("Episode reward over time")
    ax[1].set(xlabel="Episode reward over time", ylabel="Reward")
    fig.suptitle(title)

    plt.show()

In [ ]:
plot_rate(sarsa_info["length"], sarsa_info["rewards"], "GridWorld: SARSA")
plot_rate(q_info["length"], q_info["rewards"], "GridWorld: Q-Learning")

## Task 1b: Learning in Windy Grid world

WindyGridWorld is similar to GridWorld, but with a few differences. You only need to move to the target state. But this time there is a cross-wind across the center of the grid that will push you upwards. In columns 3, 4, 5, and 8 there are winds of strength 1 while in column 6 and 7 there are winds of strength 2. For more details refer Example 6.5 in

 http://incompleteideas.net/book/RLbook2020.pdf

 You only need to change the environment and reuse the SARSA and Q-learning algorithms. 

In [ ]:
#Windy Grid World environment
from WindyGridWorld import WindyGridWorld
env = WindyGridWorld()
env.reset()
env.render()

In [ ]:
num_actions = env.action_space.n 
num_states = env.observation_space.n 

print("Number of actions: ", num_actions)
print("Number of states: ", num_states)

Play around with different learning rates epsilons, and Q initializations to see what is best.

In [ ]:
num_episodes = 1000
lr = 
epsilon = 

In [ ]:
# Initialize Q function - a simplified version is used here 
# in reality the number of states may be unknown and all states may not be reachable 

# hint: use num_states as the key to a dictionary of lists
Q = 

In [ ]:
optimal_sarsa_Q, sarsa_optimal_policy, sarsa_info = sarsa(env, Q, num_actions, num_episodes, epsilon, lr)
print("\n WindyGridWorld SARSA Optimal policy: \n", sarsa_optimal_policy)

In [ ]:
optimal_Q, q_optimal_policy, q_info = q_learning(env, Q, num_actions, num_episodes, epsilon, lr)
print("\n WindyGridWorld Q-Learning Optimal policy: \n", q_optimal_policy)

In [ ]:
plot_rate(sarsa_info["length"], sarsa_info["rewards"], "GridWorld: SARSA")
plot_rate(q_info["length"], q_info["rewards"], "GridWorld: Q-Learning")

# Task 2: Analysis (Comparison of Q-learning and SARSA learning algorithms)

1. Comment on the number of episodes required to converge to the optimal policy for both environments. 
       
2. Discuss the differences in the reward graphs.  

3. Calculate the average return across the episodes for each environment. It gives a measure of the performance of the algorithm while learning (i.e., online performance).  

4. Calculate the return after convergence. It gives you a measure of the performance after the learning is completed (i.e., offline performance). 

5. Briefly summarize your results.
 
 It is advisable to rerun the algorithm a few times to get a clearer understanding of the algorithms.